In [20]:
from gcsa.event import Event
from gcsa.google_calendar import GoogleCalendar
from gcsa.recurrence import Recurrence, DAILY, SU, SA

import plotly.express as px
import plotly.graph_objects as go

import pandas as pd

from datetime import datetime, timedelta, date, time
import dateutil
from dateutil.parser import parse
import calendar
import numpy as np
from circle_cal.model import CalendarElement
import circle_cal as cc
import circle_cal.model as model
import circle_cal.plot as ccplot
import pathlib
from rich import print
import pytz
from pytz import timezone

In [57]:
ETZ = timezone("America/New_York")
def localize_any(obj, tz):
    try:
        return tz.localize(obj)
    except ValueError:
        if obj.tzinfo:
            return obj
    except AttributeError:
        return datetime.combine(obj, time(0,0), tzinfo=tz)
    return obj

In [2]:
cred = pathlib.Path('/Users/kdavis10/.config/.credentials')
cred_json = "pygooglecal.apps.googleusercontent.com.json"

try:
    gcal = GoogleCalendar(credentials_path=cred / cred_json)
except Exception as e:
    (cred / "token.pickle").unlink()
print(list(enumerate(gcal.get_calendar_list())))

[
    (0, <CalendarListEntry None - (Weather)>),
    (1, <CalendarListEntry None - (Payments)>),
    (2, <CalendarListEntry None - (Axis Ludi Calendar)>),
    (3, <CalendarListEntry None - (NYT Astronomy and Space Calendar)>),
    (4, <CalendarListEntry None - (LIPS 2015)>),
    (5, <CalendarListEntry None - (ND-DVT Provisional Shows)>),
    (6, <CalendarListEntry None - (keithwdavis@gmail.com)>),
    (7, <CalendarListEntry None - (Work)>),
    (8, <CalendarListEntry None - (Cognoscenti)>),
    (9, <CalendarListEntry None - (Academic Calendar)>),
    (10, <CalendarListEntry None - (Descriptive Astronomy)>),
    (11, <CalendarListEntry None - (DVT Student Work Schedule)>),
    (12, <CalendarListEntry None - (PTE Meal Plan (recipes))>),
    (13, <CalendarListEntry None - (LIPS 2013 Calendar (August 2 Revision))>),
    (14, <CalendarListEntry None - (indivisibleIN02 Action Calendar)>),
    (15, <CalendarListEntry None - (Clemson Planetarium)>),
    (16, <CalendarListEntry None - (Birthdays)>),
    (17, <CalendarListEntry None - (Remember The Milk)>),
    (18, <CalendarListEntry None - (Phases of the Moon)>),
    (19, <CalendarListEntry None - ( Wolverhampton)>),
    (20, <CalendarListEntry None - (Chicago Fire)>),
    (21, <CalendarListEntry None - (GenCon 2019)>),
    (22, <CalendarListEntry None - (Holidays in United States)>),
    (23, <CalendarListEntry None - (kdavis10@nd.edu)>),
    (24, <CalendarListEntry None - (DVT Student Work UNAVAILABLE)>),
    (25, <CalendarListEntry None - (Great Lakes Planetarium Association)>),
    (26, <CalendarListEntry None - (Tasks Personal)>),
    (27, <CalendarListEntry None - (Legend Secondary)>),
    (28, <CalendarListEntry None - (Legend Main)>),
    (29, <CalendarListEntry None - (TD Trips)>),
    (30, <CalendarListEntry None - (Food)>),
    (31, <CalendarListEntry None - (Tasks: Work)>),
    (32, <CalendarListEntry None - (TD Work Projects)>)
]

In [3]:
year = y24 = CalendarElement(year=2024)

traces = []

clist = list(gcal.get_calendar_list())



In [110]:
dfs = []
for cid in [6, 9, 23, 31,26]:
    events = gcal.get_events(y24.start.datetime(),
                             y24.stop.datetime(),
                             single_events=True,
                             calendar_id=clist[cid].calendar_id,
                            )

    df = pd.DataFrame(data=[model.EventWrap(ev) for ev in gcal.get_events(y24.start.datetime(),
                                                                          y24.stop.datetime())
                            if ev.other["eventType"] != "workingLocation"],
                      columns=["Event_obj"]
                     )
    

    dfs.append(df)
    
df = pd.concat(dfs, axis="rows", ignore_index=True)
df["duration"] = df["Event_obj"].apply(lambda ev: ev.duration)
df["mid"] = pd.to_datetime(df["Event_obj"].apply(lambda ev: localize_any(ev.mid, ETZ)), utc=True).dt.tz_convert(ETZ)
df["start"] = pd.to_datetime(df["Event_obj"].apply(lambda ev: localize_any(ev.start, ETZ)), utc=True).dt.tz_convert(ETZ)
df["end"] = pd.to_datetime(df["Event_obj"].apply(lambda ev: localize_any(ev.end, ETZ)), utc=True).dt.tz_convert(ETZ)
#df["end"] = df["Event_obj"].apply(lambda ev: ev.end)
df["color"] = clist[cid].background_color
df["cid"] = cid
df["calendar_id"] = clist[cid].calendar_id
df["summary"] = df["Event_obj"].apply(lambda ev: ev.summary)


In [111]:
df24 = df[df["start"] >= pd.Timestamp("2024-01-01", tzinfo=ETZ)]
df24 = df24[df24["start"] <= pd.Timestamp("2025-01-01", tzinfo=ETZ)]

In [112]:
pd.cut(df["duration"])

TypeError: cut() missing 1 required positional argument: 'bins'

In [117]:
ii = pd.IntervalIndex([pd.Interval(pd.Timedelta(days=0), pd.Timedelta(days=1)),
                       pd.Interval(pd.Timedelta(days=1), pd.Timedelta(days=1)),
                       pd.Interval(pd.Timedelta(days=1), pd.Timedelta(days=7)),
                       pd.Interval(pd.Timedelta(days=7), pd.Timedelta(days=30)),
                       pd.Interval(pd.Timedelta(days=30), pd.Timedelta(days=365))])

In [123]:
df["scale"] = pd.cut(df["duration"], ii)

In [121]:
df["scale"]

0      (0 days 00:00:00, 1 days 00:00:00]
1      (0 days 00:00:00, 1 days 00:00:00]
2      (0 days 00:00:00, 1 days 00:00:00]
3      (0 days 00:00:00, 1 days 00:00:00]
4      (0 days 00:00:00, 1 days 00:00:00]
                      ...                
470    (0 days 00:00:00, 1 days 00:00:00]
471    (0 days 00:00:00, 1 days 00:00:00]
472    (0 days 00:00:00, 1 days 00:00:00]
473    (0 days 00:00:00, 1 days 00:00:00]
474    (1 days 00:00:00, 7 days 00:00:00]
Name: scale, Length: 475, dtype: category
Categories (5, interval[timedelta64[ns], right]): [(0 days 00:00:00, 1 days 00:00:00] < (1 days 00:00:00, 1 days 00:00:00] < (1 days 00:00:00, 7 days 00:00:00] < (7 days 00:00:00, 30 days 00:00:00] < (30 days 00:00:00, 365 days 00:00:00]]

In [122]:
df

,Event_obj,duration,mid,start,end,color,cid,calendar_id,summary,scale
0,<circle_cal.model.EventWrap object at 0x10baf3...,1 days 00:00:00,2012-06-10 12:00:00-04:00,2012-06-10 00:56:00-04:00,2012-06-11 00:56:00-04:00,#96c3eb,26,bbc701bc624097396f81ecb16c5da0f62a1299bdc68ef6...,Mom's Birthday,"(0 days 00:00:00, 1 days 00:00:00]"
1,<circle_cal.model.EventWrap object at 0x10b775...,1 days 00:00:00,2012-04-08 12:00:00-04:00,2012-04-08 00:56:00-04:00,2012-04-09 00:56:00-04:00,#96c3eb,26,bbc701bc624097396f81ecb16c5da0f62a1299bdc68ef6...,Nipsco Due,"(0 days 00:00:00, 1 days 00:00:00]"
2,<circle_cal.model.EventWrap object at 0x10bae8...,1 days 00:00:00,2023-08-21 12:00:00-04:00,2023-08-21 00:56:00-04:00,2023-08-22 00:56:00-04:00,#96c3eb,26,bbc701bc624097396f81ecb16c5da0f62a1299bdc68ef6...,recycling pickup,"(0 days 00:00:00, 1 days 00:00:00]"
3,<circle_cal.model.EventWrap object at 0x10b776...,0 days 01:00:00,2024-05-07 10:30:00-04:00,2024-05-07 10:00:00-04:00,2024-05-07 11:00:00-04:00,#96c3eb,26,bbc701bc624097396f81ecb16c5da0f62a1299bdc68ef6...,Dentist,"(0 days 00:00:00, 1 days 00:00:00]"
4,<circle_cal.model.EventWrap object at 0x10b775...,0 days 01:00:00,2024-10-29 16:30:00-04:00,2024-10-29 16:00:00-04:00,2024-10-29 17:00:00-04:00,#96c3eb,26,bbc701bc624097396f81ecb16c5da0f62a1299bdc68ef6...,Dermatology,"(0 days 00:00:00, 1 days 00:00:00]"
...,...,...,...,...,...,...,...,...,...,...
470,<circle_cal.model.EventWrap object at 0x10c279...,0 days 00:30:00,2024-08-22 17:45:00-04:00,2024-08-22 17:30:00-04:00,2024-08-22 18:00:00-04:00,#96c3eb,26,bbc701bc624097396f81ecb16c5da0f62a1299bdc68ef6...,Robby Meeting,"(0 days 00:00:00, 1 days 00:00:00]"
471,<circle_cal.model.EventWrap object at 0x10c278...,0 days 01:00:00,2024-08-08 14:30:00-04:00,2024-08-08 14:00:00-04:00,2024-08-08 15:00:00-04:00,#96c3eb,26,bbc701bc624097396f81ecb16c5da0f62a1299bdc68ef6...,Therapy,"(0 days 00:00:00, 1 days 00:00:00]"
472,<circle_cal.model.EventWrap object at 0x10c27b...,0 days 01:00:00,2024-08-14 10:30:00-04:00,2024-08-14 10:00:00-04:00,2024-08-14 11:00:00-04:00,#96c3eb,26,bbc701bc624097396f81ecb16c5da0f62a1299bdc68ef6...,AC Inspection,"(0 days 00:00:00, 1 days 00:00:00]"
473,<circle_cal.model.EventWrap object at 0x10c27a...,1 days 00:00:00,2024-08-03 12:00:00-04:00,2024-08-03 00:56:00-04:00,2024-08-04 00:56:00-04:00,#96c3eb,26,bbc701bc624097396f81ecb16c5da0f62a1299bdc68ef6...,Stay Steve Alford Inn,"(0 days 00:00:00, 1 days 00:00:00]"
